# Data Analyzer

In [1]:
import os

## Hybrid Compiled, All Threads, FastMM Gradient, Bini, Dgemm, Regular

In [13]:
log_folder = 'output_all_threads_fastmm_gradient_regular_bini_dgemm/'

In [9]:
# get the prefixes for each run
types = set()
for i in os.listdir(log_folder):
    name = i.split('_')[:-1]
    fullname = ''
    for j in name:
        fullname += j + '_'
    types.add(fullname)

In [12]:
types

{'layer_4_nodes_1024_bs_1024_mm_',
 'layer_4_nodes_1024_bs_256_mm_',
 'layer_4_nodes_1024_bs_4096_mm_',
 'layer_4_nodes_1024_bs_8192_mm_',
 'layer_4_nodes_256_bs_1024_mm_',
 'layer_4_nodes_256_bs_256_mm_',
 'layer_4_nodes_256_bs_4096_mm_',
 'layer_4_nodes_256_bs_8192_mm_',
 'layer_4_nodes_4096_bs_1024_mm_',
 'layer_4_nodes_4096_bs_256_mm_',
 'layer_4_nodes_4096_bs_4096_mm_',
 'layer_4_nodes_4096_bs_8192_mm_',
 'layer_4_nodes_8192_bs_1024_mm_',
 'layer_4_nodes_8192_bs_256_mm_',
 'layer_4_nodes_8192_bs_4096_mm_',
 'layer_4_nodes_8192_bs_8192_mm_',
 'layer_8_nodes_1024_bs_1024_mm_',
 'layer_8_nodes_1024_bs_256_mm_',
 'layer_8_nodes_1024_bs_4096_mm_',
 'layer_8_nodes_1024_bs_8192_mm_',
 'layer_8_nodes_256_bs_1024_mm_',
 'layer_8_nodes_256_bs_256_mm_',
 'layer_8_nodes_256_bs_4096_mm_',
 'layer_8_nodes_256_bs_8192_mm_',
 'layer_8_nodes_4096_bs_1024_mm_',
 'layer_8_nodes_4096_bs_256_mm_'}

In [24]:
algos = ['bini322', 'dgemm', 'regular']
for t in types:
    print(f'\n\n{t}')
    for a in algos:
        try:
            f = open(f'{log_folder}{t}{a}.log', "r")
            text = f.read()
            f.close()
            avg_time_per_batch = text.split('\n')[-2].split(' ')[-1]

            print(f'{a} Average Time per batch: {avg_time_per_batch}')
        except:
            print(f'{a} Did not finish')



layer_4_nodes_4096_bs_256_mm_
bini322 Average Time per batch: 519.6261565089226
dgemm Average Time per batch: 214.87437307834625
regular Average Time per batch: 151.41955119371414


layer_4_nodes_1024_bs_4096_mm_
bini322 Average Time per batch: 30.30176877975464
dgemm Average Time per batch: 12.91502058506012
regular Average Time per batch: 8.485099375247955


layer_4_nodes_8192_bs_8192_mm_
bini322 Average Time per batch: 381.7395913004875
dgemm Average Time per batch: 283.9673982858658
regular Average Time per batch: 467.303936958313


layer_8_nodes_256_bs_256_mm_
bini322 Average Time per batch: 101.05871337652206
dgemm Average Time per batch: 14.118805706501007
regular Average Time per batch: 5.104385316371918


layer_4_nodes_8192_bs_256_mm_
bini322 Average Time per batch: 1301.804402589798
dgemm Average Time per batch: 552.1018261909485
regular Average Time per batch: 617.6278757452965


layer_4_nodes_1024_bs_8192_mm_
bini322 Average Time per batch: 22.900190353393555
dgemm Averag